In [27]:
from arcgis.gis import GIS
import geopandas as gpd
import datetime as dt

# my app link
# add credentials, api key?
user_name = 'radoslav_elias'
pwd = '6az2ziVWhjLd4X2'
gis = GIS('https://ral7dpzzhwjik5eu.maps.arcgis.com/home/item.html?id=9e81631a9db24d6fa4376dd1b4736975',
          username=user_name,
          password=pwd)

gis.users.get('radoslav_elias')

<User username:radoslav_elias>

In [30]:
cyklodetektory_item = gis.content.search('counters_matched', item_type='feature layer')[0]
type(cyklodetektory_item)


arcgis.gis.Item

In [31]:
# featureLayer object
cyklodetektory_layer = cyklodetektory_item.layers[0]

In [42]:
for f in cyklodetektory_layer.properties.fields:
    print(f['name'])

# featureSet object
query_result1 = cyklodetektory_layer.query(where="UnitId IN ('CAM1', 'CAM2')", out_fields='way_id,FirstDirection_Cyclists')

ObjectId
UnitName
UnitId
EndOfInterval
FirstDirection_Cyclists
SecondDirection_Cyclists
datum
way_id


In [43]:
query_result1.sdf

,ObjectId,way_id,FirstDirection_Cyclists,SHAPE
0,2,76821611,16,"{""x"": 1848173.2955565671, ""y"": 6306088.8718656..."
1,4,83202890,26,"{""x"": 1848163.7408933416, ""y"": 6306079.3172465..."
2,36,76821611,63,"{""x"": 1848173.2955565671, ""y"": 6306088.8718656..."
3,42,83202890,0,"{""x"": 1848163.7408933416, ""y"": 6306079.3172465..."
4,43,83202890,7,"{""x"": 1848163.7408933416, ""y"": 6306079.3172465..."
...,...,...,...,...
10813,78022,83202890,0,"{""x"": 1848163.7408933416, ""y"": 6306079.3172465..."
10814,78033,76821611,0,"{""x"": 1848173.2955565671, ""y"": 6306088.8718656..."
10815,78034,83202890,0,"{""x"": 1848163.7408933416, ""y"": 6306079.3172465..."
10816,78045,76821611,0,"{""x"": 1848173.2955565671, ""y"": 6306088.8718656..."


In [45]:
# parse unique way_ids from query result
way_ids = [feature.attributes['way_id'] for feature in query_result1.features]
tuple(set(way_ids))

(83202890, 76821611)

In [50]:
# query from strava layer by way_ids found in counters data
strava_item = gis.content.search('model', item_type='feature layer')[0]
strava_layer = strava_item.layers[0]
query_result2 = strava_layer.query(where="osm_reference_id IN {}".format(tuple(set(way_ids))), out_fields="edge_uid")
query_result2.sdf

,ObjectId,edge_uid,SHAPE
0,16225,161058891,"{""paths"": [[[1848187.81262003, 6306078.7800954..."
1,53736,161058894,"{""paths"": [[[1848133.06569446, 6306114.8513829..."


In [6]:
# upload to layer - not currently working
"""gdf = gpd.read_file('datasets/do_prace_na_kole.geojson')
dpnk_props = {"title":"DoPraceNaKole_" + str(int(dt.datetime.now().timestamp())),
                        "type":"File Geodatabase",
                        "tags":"Brno, Do prace na kole",
                        "snippet":"Data from bike-to-work event aggregated to street sections",
                        "description":"Data downloaded from data.brno.cz"}

if not "datasets" in [folder['title'] for folder in gis.users.me.folders]:
    gis.content.create_folder("datasets")

dpnk_fgdb_item = gis.content.add(item_properties=dpnk_props,
                                           data=gdf,
                                           folder="datasets")
dpnk_item = dpnk_fgdb_item.publish()"""


AttributeError: Cannot use 'geom' accessor on objects of dtype 'geometry'.